<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [13]:
## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [27]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
import pandas as pd
import folium

# Read the CSV file into a pandas DataFrame
spacex_df = pd.read_csv('spacex_launch_geo.csv')

# Display the first few rows of the DataFrame to understand its structure
print(spacex_df.head())

# Initial the map centered around the mean latitude and longitude of the launch sites
map_center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# For each launch site, add a Circle and Marker object based on its coordinate (Lat, Long) values
for _, row in spacex_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#0000FF',
        fill=True,
        fill_color='#0000FF',
        popup=folium.Popup(row['Launch Site'], parse_html=True)
    ).add_to(site_map)
    
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.DivIcon(html=f'''
            <div style="font-size: 12; color: #d35400;">
                <b>{row['Launch Site']}</b>
            </div>
        ''')
    ).add_to(site_map)

# Save the map as an HTML file and display it
site_map.save('spacex_launch_sites_map.html')
site_map

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


### Analysis of SpaceX Launch Sites

#### 1. Proximity to the Equator Line:
The equator is located at 0 degrees latitude. Launch sites closer to the equator are preferred for equatorial orbits and for taking advantage of the Earth's rotational speed to provide an extra boost to the rocket.

- **Vandenberg Air Force Base (VAFB)**: Located at approximately 34.6°N latitude, it is not very close to the equator.
- **Space Launch Complex 40 (SLC-40)**: Located at approximately 28.5°N latitude, it is closer to the equator than VAFB.
- **Launch Complex 39A (LC-39A)**: Located at approximately 28.6°N latitude, it is also relatively close to the equator.
- **Kennedy Space Center (KSC)**: Also located at approximately 28.6°N latitude.

#### 2. Proximity to the Coast:
Launch sites are often located near the coast to allow rockets to launch over the ocean, minimizing the risk to populated areas in case of a launch failure.

- **Vandenberg Air Force Base (VAFB)**: Located very close to the Pacific Ocean.
- **Space Launch Complex 40 (SLC-40)**: Located near the coast of Florida, close to the Atlantic Ocean.
- **Launch Complex 39A (LC-39A)**: Also near the coast of Florida, close to the Atlantic Ocean.
- **Kennedy Space Center (KSC)**: Located very close to the Atlantic Ocean.

### Findings:
1. **Proximity to the Equator**:
   - Not all launch sites are in close proximity to the equator. The launch sites in Florida (SLC-40, LC-39A, KSC) are relatively closer to the equator compared to the Vandenberg Air Force Base in California.

2. **Proximity to the Coast**:
   - All launch sites are in very close proximity to the coast. This allows rockets to be launched over the ocean, which is a safer option compared to launching over populated areas.

In [15]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [17]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [30]:
print(spacex_df)


    Flight Number        Date Time (UTC) Booster Version   Launch Site  \
0               1  2010-06-04   18:45:00  F9 v1.0  B0003   CCAFS LC-40   
1               2  2010-12-08   15:43:00  F9 v1.0  B0004   CCAFS LC-40   
2               3  2012-05-22    7:44:00  F9 v1.0  B0005   CCAFS LC-40   
3               4  2012-10-08    0:35:00  F9 v1.0  B0006   CCAFS LC-40   
4               5  2013-03-01   15:10:00  F9 v1.0  B0007   CCAFS LC-40   
5               7  2013-12-03   22:41:00         F9 v1.1   CCAFS LC-40   
6               8  2014-01-06   22:06:00         F9 v1.1   CCAFS LC-40   
7               9  2014-04-18   19:25:00         F9 v1.1   CCAFS LC-40   
8              10  2014-07-14   15:15:00         F9 v1.1   CCAFS LC-40   
9              11  2014-08-05    8:00:00         F9 v1.1   CCAFS LC-40   
10             12  2014-09-07    5:00:00   F9 v1.1 B1011   CCAFS LC-40   
11             13  2014-09-21    5:52:00   F9 v1.1 B1010   CCAFS LC-40   
12             14  2015-01-10    9:47:

In [32]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


# Function to assign colors based on class value
def assign_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the dataframe to check the new column
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long,marker_color
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356,red
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356,red
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356,red
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356,red
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [33]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [34]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
# Create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successful or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color'])
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}\nClass: {record['class']}"
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [20]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [35]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [36]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [23]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [24]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [37]:
# Given coordinates of the closest coastline
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance using the given function
launch_site_lat = 28.563197
launch_site_lon = -80.576820

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a folium.Marker on your selected closest coastline point on the map
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_coastline),
    )
)

site_map.add_child(distance_marker)

# Display the map with the new marker
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [38]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

<class 'NameError'>: name 'lines' is not defined

In [40]:
# Coordinates of the launch site and the coastline point
launch_site_coord = [28.563197, -80.576820]
coastline_coord = [28.56367, -80.57163]

# Create a folium.PolyLine object using the coastline coordinates and launch site coordinates
lines = folium.PolyLine(locations=[launch_site_coord, coastline_coord], weight=1)

# Add the PolyLine to the site_map
site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [42]:
from math import sin, cos, sqrt, atan2, radians
import folium

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Coordinates of the launch site
launch_site_coord = [28.563197, -80.576820]

# Coordinates of the closest city, railway, highway, etc.
closest_city_coord = [28.0836, -80.6081] # Example coordinates for Melbourne, FL
closest_railway_coord = [28.5721, -80.5853] # Example coordinates for NASA Railroad
closest_highway_coord = [28.5640, -80.5708] # Example coordinates for Samuel C Phillips Pkwy

# Calculate distances
distance_city = calculate_distance(launch_site_coord[0], launch_site_coord[1], closest_city_coord[0], closest_city_coord[1])
distance_railway = calculate_distance(launch_site_coord[0], launch_site_coord[1], closest_railway_coord[0], closest_railway_coord[1])
distance_highway = calculate_distance(launch_site_coord[0], launch_site_coord[1], closest_highway_coord[0], closest_highway_coord[1])

# Create a map
site_map = folium.Map(location=launch_site_coord, zoom_start=10)

# Create markers for the closest city, railway, and highway
city_marker = folium.Marker(
    location=closest_city_coord,
    popup=f"Closest City: Melbourne, FL<br>Distance: {distance_city:.2f} km",
    icon=folium.Icon(color='blue')
)
railway_marker = folium.Marker(
    location=closest_railway_coord,
    popup=f"Closest Railway: NASA Railroad<br>Distance: {distance_railway:.2f} km",
    icon=folium.Icon(color='blue')
)
highway_marker = folium.Marker(
    location=closest_highway_coord,
    popup=f"Closest Highway: Samuel C Phillips Pkwy<br>Distance: {distance_highway:.2f} km",
    icon=folium.Icon(color='blue')
)

# Add markers to the map
city_marker.add_to(site_map)
railway_marker.add_to(site_map)
highway_marker.add_to(site_map)

# Draw lines between the launch site and the markers
folium.PolyLine(locations=[launch_site_coord, closest_city_coord], color='blue', weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine(locations=[launch_site_coord, closest_railway_coord], color='blue', weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine(locations=[launch_site_coord, closest_highway_coord], color='blue', weight=2.5, opacity=1).add_to(site_map)

# Display the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


Based on the map and the plotted distance lines, we can answer the following questions:

1. **Are launch sites in close proximity to railways?**
   - Yes, the launch sites are often located near railways. This proximity facilitates the transportation of heavy equipment and materials needed for rocket launches.

2. **Are launch sites in close proximity to highways?**
   - Yes, launch sites are also close to highways. This accessibility is crucial for the transportation of personnel, supplies, and equipment to the launch site efficiently.

3. **Are launch sites in close proximity to coastline?**
   - Yes, launch sites are typically located near coastlines. Coastal locations are preferred to ensure that any malfunctioning rockets or debris fall into the ocean, minimizing the risk to populated areas.

4. **Do launch sites keep a certain distance away from cities?**
   - Yes, launch sites are generally kept at a safe distance from cities. This distance helps to mitigate the risks associated with rocket launches, such as potential explosions or falling debris, ensuring the safety of the population.

These considerations are critical in the site selection process for space launch facilities, balancing the needs for safety, transportation logistics, and minimizing risks to populated areas.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
